<a href="https://colab.research.google.com/github/IvanDePivan/2AMM30-groep-2-component-1/blob/tamara/cleaning_nobel_articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install ftfy
!pip install srsly

In [5]:
from pathlib import Path
import ftfy
import random
import srsly
import os
import re
from tqdm import tqdm

In [6]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Get email of current Colab user
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
email = gcloud_tokeninfo['email']

In [8]:
# Define notebook directory
if email == 'tamaraexterkate93@gmail.com':
  notebook_dir = "/content/drive/MyDrive/TUe/TM/Data/enwiki20230820-stripped-json"
else:
  notebook_dir = 'drive/MyDrive/enwiki20230820/raw/'

print(notebook_dir)

/content/drive/MyDrive/TUe/TM/Data/enwiki20230820-stripped-json


In [9]:
# Generate a list of file paths
if email == 'tamaraexterkate93@gmail.com':
  pathlist = [os.path.join(root, file) for root, dirs, files in os.walk(notebook_dir) for file in files]
else:
  pathlist = [p for p in Path(notebook_dir).glob('**/*') if p.is_file()]

In [10]:
# Search terms
search_terms = ['nobel', 'prize']

In [11]:
# Read files using srsly (generators)
from itertools import chain

def get_json_readers():
  return chain.from_iterable(srsly.read_jsonl(path) for path in pathlist)

def read_random_json():
  data = srsly.read_jsonl(random.choice(pathlist))
  articles = [article for article in data if len(article["text"]) > 0]
  articles = [article for article in articles if (re.search(term, article["text"], re.IGNORECASE) for term in search_terms)]

  return articles

def read_all_json():
  data = get_json_readers()
  articles = [article for article in data if len(article["text"]) > 0 and all(re.search(term, article["text"], re.IGNORECASE) is not None for term in search_terms)]
  print(f'found {len(articles)} nonempty nobel prize laureate articles')
  return articles

def read_all_json_owen():
  data = get_json_readers()
  articles = [article for article in data if "Owen Willans Richardson" in article["title"]]
  print(f'found {len(articles)} nonempty nobel prize laureate articles')
  return articles

def read_all_json_title():
  data = get_json_readers()
  articles = [article for article in data if all(re.search(term, article["title"], re.IGNORECASE) is not None for term in search_terms)]
  print(f'found {len(articles)} nonempty nobel prize laureate articles')
  return articles

In [12]:
def clean_text(text):
  # Note: the order of these regexes matter, due to '\n' -> ' ' for instance resulting in repeated spaces
  url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


  ordered_list = re.compile(r'^[A-Z0-9]{1,2}\.', re.MULTILINE)
  cleaned_text = ftfy.fix_text(text)
  cleaned_text = re.sub('\(([;,] ?)+\)', '\(', cleaned_text) # Removes artifacts from within parentheses e.g. (; ; born in etc) -> (born in etc)
  cleaned_text = re.sub('\(.?\)', '', cleaned_text) # Remove empty or 1 character parentheses e.g. "( )"
  cleaned_text = re.sub('\n', ' ', cleaned_text) # Remove newlines
  cleaned_text = re.sub(ordered_list, '', cleaned_text) # Removes the first part of an ordered list (e.g. A. the cheese ->  the cheese)
  cleaned_text = re.sub(' {2,}', ' ', cleaned_text) # Remove repeated spaces
  cleaned_text = re.sub('[“”]', '"', cleaned_text) # Remove smart double quotes, might also need one for smart single quotes
  cleaned_text = re.sub('([12]\d)(\d{2})[–\/](\d{2}\D)', r'\1\2-\1\3', cleaned_text)
  cleaned_text = re.sub(url_pattern, '', cleaned_text)

  return cleaned_text

In [13]:
# # Not needed if you have the .json
# articles = read_all_json()
# srsly.write_json(os.path.join(notebook_dir, 'nobel_articles.json'), articles)

In [31]:
articles = srsly.read_json(os.path.join(notebook_dir, 'nobel_articles.json'))
print(len(articles))

14980


In [28]:
term = "Physics|Chemistry|Physiology|Medicine|Literature|Peace"
articles = [article for article in articles if re.search(term, article["text"], re.IGNORECASE) is not None]
print(len(articles))

11893


In [29]:
term = "born"
articles = [article for article in articles if re.search(term, article["text"], re.IGNORECASE) is not None]
print(len(articles))

6484


In [30]:
term = "awarded|laureate"
articles = [article for article in articles if re.search(term, article["text"], re.IGNORECASE) is not None]
print(len(articles))

3917


In [26]:
titles = [article['title'] for article in articles]
for title in sorted(titles):
  print(title)

Streaminguitvoer ingekort tot de laatste 5000 regels.
Donald S. Russell
Donald Sadoway
Donald Seldin
Donald Tusk
Donna Strickland
Donna Summer
Donovan Webster
Dopamine
Dora Melegari
Dora Staudinger
Doreen Nabwire
Doris Lessing
Dorothy Cotton
Dorothy Donaldson Buchanan
Dorothy Hodgkin
Dorothy M. Horstmann
Dorothy Thompson
Dottie Thomas
Doug Walgren
Douglas Hofstadter
Douglas MacDiarmid
Douglas Murray (author)
Douglas Osheroff
Douglas Prasher
Douglas Youvan
Douglass North
Dragan Šormaz
Draper Laboratory
Drenka Willen
Drew Denbaum
Drew Gilpin Faust
Drew University
Dril
Drug Enforcement Administration
Duane Silverstein
Dudley R. Herschbach
Duke Energy
Dulce María Loynaz
Dumitru Mazilu
Duncan Haldane
Dutch Americans
Dwijendranath Tagore
E. C. George Sudarshan
E. D. Morel
E. Donnall Thomas
E. M. Forster
E. P. Thompson
E. Stanley Jones
ETA (separatist group)
Ear
Earl Wilbur Sutherland Jr.
Earth Day
East Asia–United States relations
East End literature
East Midlands
East Timor
Eastbourne
Ebene

In [16]:
def find_nobel_winners(text):
    str_length = len(text)
    if str_length < 600:
        return True
    else:
        length = max(600, 0.2*str_length)
        new_text = text[:int(length)]
        return all(re.search(term, new_text, re.IGNORECASE) is not None for term in search_terms)

In [17]:
articles = [article for article in articles if find_nobel_winners(article['text'])]
print(len(articles))

4341


In [18]:
#word 'nobel' has to be in it 3 or more times
articles = [article for article in get_json_readers() if len(re.findall(r'nobel', article["text"], re.IGNORECASE)) >= 3]
print(len(articles))

ValueError: ignored

In [ ]:
articles_cleaned = [{key: (clean_text(value) if key == "text" else value) for key, value in article.items()} for article in articles]

In [ ]:
# write cleaned articles to file
srsly.write_json(os.path.join(notebook_dir, 'nobel_articles_cleaned.json'), articles_cleaned)

In [ ]:
titles = [article['title'] for article in articles]
for title in sorted(titles):
  print(title)
